# Introduction
Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image.
n the 3-dimensional space is that described in: [K. P. Bennett and O. L. Mangasarian: "Robust Linear Programming Discrimination of Two Linearly Inseparable Sets", Optimization Methods and Software 1, 1992, 23-34].

This database is also available through the UW CS ftp server:
ftp ftp.cs.wisc.edu
cd math-prog/cpo-dataset/machine-learn/WDBC/

Also can be found on UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

# Content:
[1. Load and Check Data](#1)
[1. Variable Description](#2)
    [1. Univariate Variable Analysis](#3)
        [1. Categorical Variable](#4)
        [1. Numerical Variable](#5)
[1. Basic Data Analysis](#6)
[1. Outlier Detection](#7)
[1. Missing Value](#8)
    [1. Find Missing Value](#9)
    [1. Fill Missing Value](#10)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
#plt.style.available

import seaborn as sns
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 <a id='1'></a>
# Load and Check Data

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.describe

 <a id='2'></a>
# Variable Description
Attribute Information:

1) ID number
2) Diagnosis (M = malignant, B = benign)
3-32)

Ten real-valued features are computed for each cell nucleus:

a) radius (mean of distances from center to points on the perimeter)
b) texture (standard deviation of gray-scale values)
c) perimeter
d) area
e) smoothness (local variation in radius lengths)
f) compactness (perimeter^2 / area - 1.0)
g) concavity (severity of concave portions of the contour)
h) concave points (number of concave portions of the contour)
i) symmetry
j) fractal dimension ("coastline approximation" - 1)

The mean, standard error and "worst" or largest (mean of the three
largest values) of these features were computed for each image,
resulting in 30 features. For instance, field 3 is Mean Radius, field
13 is Radius SE, field 23 is Worst Radius.

All feature values are recoded with four significant digits.

Missing attribute values: none

Class distribution: 357 benign, 212 malignant

In [ ]:
df.info()

 <a id='3'></a>
# Univariate Variable Analysis
* Categorical Variable: diagnosis, 
* Numerical Variable: radius_mean,texture_mean, perimeter_mean, area_mean, smoothness_mean, compactness_mean, concavity_mean, concave points_mean, symmetry_mean, fractal_dimension_mean, perimeter_se, area_se, smoothness_se, compactness_se, concavity_se, concave points_se, symmetry_se, fractal_dimension_se, perimeter_worst, area_worst, smoothness_worst, compactness_worst, concavity_worst, concave points_worst, symmetry_worst, fractal_dimension_worst

 <a id='4'></a>
# Categorical Variable:

In [ ]:
def bar_plot(variable):
    """
        input: variable ex: "Diagnosis"
        output: bar plot & value count
    """
    # get feature
    var = df[variable]
    # count number of categorical variable(value/sample)
    varValue = var.value_counts()
    
    # visualize
    plt.figure(figsize = (9,3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n {}".format(variable,varValue))

In [ ]:
category1 = ["diagnosis"]
for c in category1:
    bar_plot(c)

 <a id='5'></a>
# Numerical Variable:

In [ ]:
def plot_hist(variable):
    plt.figure(figsize=(9,3))
    plt.hist(df[variable],bins=10)
    #plt.hist(df[variable],bins=890)
    """We need to increase the value of bins to examine the graf in e little more detail"""
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} distribution with hist".format(variable))
    plt.show()

In [ ]:
numericVar1=["perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", "fractal_dimension_mean"]
for n in numericVar1:
    plot_hist(n)

 <a id='6'></a>
# Basic Data Analysis
the relationship of all areas with diagnosis area

In [ ]:
# radius_mean vs diagnosis
df[["radius_mean","diagnosis"]].groupby(["radius_mean"],as_index=False).count()
#df[["radius_mean","diagnosis"]].groupby(["radius_mean"],as_index=False).mean().sort_values(by="diagnosis",ascending=False)

 <a id='7'></a>
# Outlier Detection

In [ ]:
def detect_outliers(dfl, features):
    outlier_indices=[]
    
    for c in features:
        #1st quartile
        Q1 = np.percentile(dfl[c],25)
        #3rd quartile
        Q3 = np.percentile(dfl[c],75)
        #IQR
        IQR = Q3 - Q1
        #Outlier step
        outlier_step = IQR * 1.5
        #detect outlier and their indices
        outlier_list_col = dfl[(dfl[c] < Q1 - outlier_step) | (dfl[c] > Q3 + outlier_step)].index
        # store indices
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices) 
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    return multiple_outliers

In [ ]:
df.loc[detect_outliers(df,["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean"])]

In [ ]:
# drop Outliers
df = df.drop(detect_outliers(df,["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean"]),axis=0).reset_index(drop=True)

 <a id='8'></a>
# Missing Value
* Find Missing Value
* Fill Missing Value

In [ ]:
df.head()

 <a id='9'></a>
# Find Missing Value

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
df.isnull().sum()